# Experiment 1

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Imports

In [2]:
import gc
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch import nn, optim
from torchinfo import summary

from plant_village_dataset import PlantVillageDataset
from runner import Runner
from mlp import MLP
from convnext import ConvNext
from encoder_mlp import EncoderMLP
from unet_autoencoder import UNetAutoencoder, UNetEncoder, UNetDecoder

## Prepare Data

In [41]:
DEVICE = 'mps'

In [42]:
BATCH_SIZE = 64

In [43]:
def split(dataset, batch_size, labeled_ratio, test_ratio):    
    labels = dataset.get_labels()
    labels = np.array(labels)

    unlabeled_indices, labeled_indices = train_test_split(np.arange(len(dataset)),
                                                          test_size=labeled_ratio,
                                                          stratify=labels)   
    
    ul_train_indices, ul_val_indices = train_test_split(unlabeled_indices, test_size=0.1)
    
    relative_test_ratio = test_ratio / labeled_ratio
    
    train_val_indices, test_indices = train_test_split(labeled_indices,
                                                       test_size=relative_test_ratio,
                                                       stratify=labels[labeled_indices])
    
    train_indices, val_indices = train_test_split(train_val_indices,
                                                  test_size=0.2,
                                                  stratify=labels[train_val_indices])

    ul_train_sampler = SubsetRandomSampler(ul_train_indices)
    ul_val_sampler = SubsetRandomSampler(ul_val_indices)
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    ul_train_loader = DataLoader(dataset, batch_size=batch_size, sampler=ul_train_sampler, num_workers=12)
    ul_val_loader = DataLoader(dataset, batch_size=batch_size, sampler=ul_val_sampler, num_workers=12)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=12)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler, num_workers=12)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler, num_workers=12)

    return ul_train_loader, ul_val_loader, train_loader, val_loader, test_loader

In [44]:
class ReconstructionDataLoader:
    def __init__(self, base_loader):
        self.base_loader = base_loader

    def __iter__(self):
        for data in self.base_loader:
            images, _ = data  # Ignore labels or other types of data
            yield images, images  # Yield images as both input and target

    def __len__(self):
        return len(self.base_loader)

In [3]:
dataset = PlantVillageDataset('images')

Loading Plant Village


/Users/ariel.arevalo/Workspace/ci0148/proyecto_2/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


 - Normalizing dataset


 - Calculating mean and standard deviation:   0%|          | 0/867 [00:00<?, ?batch/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ariel.arevalo/Workspace/ci0148/proyecto_2/venv/lib/python3.11/site-packages/torch/__init__.py", line 1896, in <module>
    from torch import export as export
  File "/Users/ariel.arevalo/Workspace/ci0148/proyecto_2/venv/lib/python3.11/site-packages/torch/export/__init__.py", line 64, in <module>
 - Calculating mean and standard deviation:  

KeyboardInterrupt: 

In [46]:
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

## Run 1

In [47]:
ul_train_loader, ul_val_loader, train_loader, val_loader, test_loader = split(dataset, batch_size=BATCH_SIZE, labeled_ratio=0.2, test_ratio=0.1)

ul_train_loader = ReconstructionDataLoader(ul_train_loader)
ul_val_loader = ReconstructionDataLoader(ul_val_loader)

##### CNN

In [48]:
cnn = ConvNext(num_classes=len(dataset.classes))
cnn_optim = optim.Adam(cnn.parameters(), lr=1e-4)
cnn_criterion = nn.CrossEntropyLoss()
cnn_runner = Runner('cnn_1', cnn, cnn_optim, cnn_criterion, DEVICE)
cnn_runner.train(train_loader, val_loader, num_epochs=3)
cnn_runner.test(test_loader, idx_to_class)

Training:   0%|          | 0/3 [00:00<?, ? epoch/s]

Training:   0%|          | 0/70 [00:00<?, ?batch/s]

Validating:   0%|          | 0/18 [00:00<?, ?batch/s]

Epoch 1/3 - Train Loss: 2.0144, Validation Loss: 0.9056


Training:   0%|          | 0/70 [00:00<?, ?batch/s]

KeyboardInterrupt: 

##### Autoencoder

In [ ]:
uae = UNetAutoencoder()
uae_optim = optim.Adam(uae.parameters(), lr=1e-3)
uae_criterion = nn.MSELoss()
uae_runner = Runner('uae_1', uae, uae_optim, uae_criterion, DEVICE)
uae_runner.train(ul_train_loader, ul_val_loader, num_epochs=3)

enc = uae.encoder

##### Frozen Encoder + MLP

In [ ]:
mlp = MLP(input_size=512, hidden_sizes=[2048, 1024, 512, 256], output_size=len(dataset.classes), dropout_rate=0.2)
emlp = EncoderMLP(encoder=enc, mlp=mlp)
emlp_optim = optim.Adam(emlp.parameters(), lr=1e-4)
emlp_criterion = nn.CrossEntropyLoss()
emlp_runner = Runner('emlp_1_a', emlp, emlp_optim, emlp_criterion, DEVICE)
emlp.freeze_encoder()
emlp_runner.train(train_loader, val_loader, num_epochs=15)
emlp_runner.test(test_loader, idx_to_class)

##### Live Encoder + MLP

In [ ]:
mlp = MLP(input_size=512, hidden_sizes=[2048, 1024, 512, 256], output_size=len(dataset.classes), dropout_rate=0.2)
emlp = EncoderMLP(encoder=enc, mlp=mlp)
emlp_optim = optim.Adam(emlp.parameters(), lr=1e-4)
emlp_criterion = nn.CrossEntropyLoss()
emlp_runner = Runner('emlp_1_b', emlp, emlp_optim, emlp_criterion, DEVICE)
emlp.unfreeze_encoder()
emlp_runner.train(train_loader, val_loader, num_epochs=15)
emlp_runner.test(test_loader, idx_to_class)

## Run 2

In [ ]:
ul_train_loader, ul_val_loader, train_loader, val_loader, test_loader = split(dataset, batch_size=BATCH_SIZE, labeled_ratio=0.5, test_ratio=0.15)

ul_train_loader = ReconstructionDataLoader(ul_train_loader)
ul_val_loader = ReconstructionDataLoader(ul_val_loader)

##### CNN

In [ ]:
cnn = ConvNext(num_classes=len(dataset.classes))
cnn_optim = optim.Adam(cnn.parameters(), lr=1e-4)
cnn_criterion = nn.CrossEntropyLoss()
cnn_runner = Runner('cnn_2', cnn, cnn_optim, cnn_criterion, DEVICE)
cnn_runner.train(train_loader, val_loader, num_epochs=3)
cnn_runner.test(test_loader, idx_to_class)

##### Autoencoder

In [ ]:
uae = UNetAutoencoder()
uae_optim = optim.Adam(uae.parameters(), lr=1e-3)
uae_criterion = nn.MSELoss()
uae_runner = Runner('uae_2', uae, uae_optim, uae_criterion, DEVICE)
uae_runner.train(ul_train_loader, ul_val_loader, num_epochs=3)

enc = uae.encoder

##### Frozen Encoder + MLP

In [ ]:
mlp = MLP(input_size=512, hidden_sizes=[2048, 1024, 512, 256], output_size=len(dataset.classes), dropout_rate=0.2)
emlp = EncoderMLP(encoder=enc, mlp=mlp)
emlp_optim = optim.Adam(emlp.parameters(), lr=1e-4)
emlp_criterion = nn.CrossEntropyLoss()
emlp_runner = Runner('emlp_2_a', emlp, emlp_optim, emlp_criterion, DEVICE)
emlp.freeze_encoder()
emlp_runner.train(train_loader, val_loader, num_epochs=15)
emlp_runner.test(test_loader, idx_to_class)

##### Live Encoder + MLP

In [ ]:
mlp = MLP(input_size=512, hidden_sizes=[2048, 1024, 512, 256], output_size=len(dataset.classes), dropout_rate=0.2)
emlp = EncoderMLP(encoder=enc, mlp=mlp)
emlp_optim = optim.Adam(emlp.parameters(), lr=1e-4)
emlp_criterion = nn.CrossEntropyLoss()
emlp_runner = Runner('emlp_2_b', emlp, emlp_optim, emlp_criterion, DEVICE)
emlp.unfreeze_encoder()
emlp_runner.train(train_loader, val_loader, num_epochs=15)
emlp_runner.test(test_loader, idx_to_class)